# Chunker 

## Setup Environment

In [6]:
!pip install langchain

In [3]:
import pandas as pd

## Get Data

In [34]:
# Read from pikle file
webpages_df = pd.read_pickle('./data/webpages_raw.pkl')

webpages_df.head(5)

,content,banner_title,banner_divisions,intro,toc,url,scrape_date
0,\n## Waarom willen de kinderartsen dat je baby...,24 uur observatie van de pasgeborene,[{'division_url': 'https://www.azstlucas.be/sp...,Elke pasgeborene wordt - bij overnachting - bi...,[{'link_url': '#waarom-willen-we-dat-je-baby-m...,https://www.azstlucas.be/onderzoek-en-behandel...,12/11/2023 15:26:50
1,\nWe weten dat reflux en opboeren voor een aan...,24 uur ph-metrie meting,[{'division_url': 'https://www.azstlucas.be/sp...,Een ph-metrie meting met impedantiemeting is e...,"[{'link_url': '#hoe-gebeurt-de-meting', 'link_...",https://www.azstlucas.be/onderzoek-en-behandel...,12/11/2023 15:26:50
2,\n## Algemene richtlijnen\n\n\nIn opdracht van...,24 uur stoelgang sparen: richtlijnen,[{'division_url': 'https://www.azstlucas.be/sp...,,"[{'link_url': '#algemene-richtlijnen', 'link_t...",https://www.azstlucas.be/onderzoek-en-behandel...,12/11/2023 15:26:51
3,\n## Algemene richtlijnen\n\n\nIn opdracht van...,24 uur urine sparen,[{'division_url': 'https://www.azstlucas.be/sp...,,"[{'link_url': '#algemene-richtlijnen', 'link_t...",https://www.azstlucas.be/onderzoek-en-behandel...,12/11/2023 15:26:52
4,\nBij een 24-uurs bloeddrukmeting wordt de blo...,24-uurs bloeddrukmeting,[{'division_url': 'https://www.azstlucas.be/sp...,Als je last hebt van hoge bloeddruk of hyperte...,[],https://www.azstlucas.be/onderzoek-en-behandel...,12/11/2023 15:26:52


## Preprocessing

Get single string content (to be chunked before embedding)

In [6]:
def combine_content(df: pd.DataFrame) -> pd.DataFrame:
    """
        # page_template: str = f"""
        # PAGE_TITLE: {}
        # PAGE_AREAS: ..., ...    
    """
    page_template: str = f"""
        PAGE_TITLE: {banner_title}
    """
    pass

webpages_processed_df = (
    webpages_df
    .pipe(combine_content)
)

### Chunk Content

In [32]:
from typing import List
from langchain.text_splitter import (
    CharacterTextSplitter, 
    RecursiveCharacterTextSplitter, 
    MarkdownHeaderTextSplitter
)

text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
    ("####", "Header 4"),
]

# Split
# splits = text_splitter.split_documents(md_header_splits)


def content_chunker(s: str, size=250, overlap=30) -> List[object]: 
    # MD splits
    markdown_splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=headers_to_split_on
    )
    md_header_splits = markdown_splitter.split_text(
        s
    )
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=size, 
        chunk_overlap=overlap
    )
    chunks = text_splitter.split_text(s)
    return chunks
    

In [44]:
def chunk_content(df: pd.DataFrame, s:int, o:int) -> pd.DataFrame: 
    df[f'chunk_s{s}_o{o}'] = df.content.apply(
        content_chunker,
        size=s, 
        overlap=o
    )

    # df['chunk_s500_o60'] = df.content.apply(
    #     content_chunker,
    #     size=500, 
    #     overlap=60
    # )
    return df 

def add_order(df: pd.DataFrame) -> pd.DataFrame: 
    df['chunk_order'] = df.reset_index().index
    return df 

chunks_df = (
    webpages_df.copy(deep=True)
    .pipe(chunk_content, s=500, o=60)  # Size & Overlap
    .pipe(pd.DataFrame.explode, column='chunk_s500_o60')
    .pipe(add_order)
)

chunks_df.head()


,content,banner_title,banner_divisions,intro,toc,url,scrape_date,chunk_s500_o60,chunk_order
0,\n## Waarom willen de kinderartsen dat je baby...,24 uur observatie van de pasgeborene,[{'division_url': 'https://www.azstlucas.be/sp...,Elke pasgeborene wordt - bij overnachting - bi...,[{'link_url': '#waarom-willen-we-dat-je-baby-m...,https://www.azstlucas.be/onderzoek-en-behandel...,12/11/2023 15:26:50,## Waarom willen de kinderartsen dat je baby m...,0
0,\n## Waarom willen de kinderartsen dat je baby...,24 uur observatie van de pasgeborene,[{'division_url': 'https://www.azstlucas.be/sp...,Elke pasgeborene wordt - bij overnachting - bi...,[{'link_url': '#waarom-willen-we-dat-je-baby-m...,https://www.azstlucas.be/onderzoek-en-behandel...,12/11/2023 15:26:50,## Mogelijke afwijkingen\n\n\n### Aangeboren h...,1
0,\n## Waarom willen de kinderartsen dat je baby...,24 uur observatie van de pasgeborene,[{'division_url': 'https://www.azstlucas.be/sp...,Elke pasgeborene wordt - bij overnachting - bi...,[{'link_url': '#waarom-willen-we-dat-je-baby-m...,https://www.azstlucas.be/onderzoek-en-behandel...,12/11/2023 15:26:50,### Infecties\n\n\nVerschillende infecties wor...,2
0,\n## Waarom willen de kinderartsen dat je baby...,24 uur observatie van de pasgeborene,[{'division_url': 'https://www.azstlucas.be/sp...,Elke pasgeborene wordt - bij overnachting - bi...,[{'link_url': '#waarom-willen-we-dat-je-baby-m...,https://www.azstlucas.be/onderzoek-en-behandel...,12/11/2023 15:26:50,### Aangeboren darmafwijkingen\n\n\nHet is pas...,3
0,\n## Waarom willen de kinderartsen dat je baby...,24 uur observatie van de pasgeborene,[{'division_url': 'https://www.azstlucas.be/sp...,Elke pasgeborene wordt - bij overnachting - bi...,[{'link_url': '#waarom-willen-we-dat-je-baby-m...,https://www.azstlucas.be/onderzoek-en-behandel...,12/11/2023 15:26:50,De kinderarts onderzoekt je baby in normale om...,4


In [46]:
chunks_df.iloc[0]['chunk_s500_o60']

"## Waarom willen de kinderartsen dat je baby minstens 24 uur op de dienst verblijft?\n\nGelukkig worden de meeste baby's helemaal gezond geboren na een normale zwangerschap. Toch is het belangrijk om een pasgeborene de eerste 24 levensuren bij ons in het ziekenhuis te observeren. Er ontstaan soms problemen die kort na de geboorte nog niet duidelijk zijn.\n\n\n\n## Mogelijke afwijkingen\n\n\n### Aangeboren hartafwijking"

### Serialize Chunks

In [47]:
# Save DF to Blob Storage for later retrieval. 
chunks_df.to_pickle(path='./data/chunks_raw.pkl')

In [48]:
# Read from pikle file
chunks_test_df = pd.read_pickle('./data/chunks_raw.pkl')

chunks_test_df.head()

,content,banner_title,banner_divisions,intro,toc,url,scrape_date,chunk_s500_o60,chunk_order
0,\n## Waarom willen de kinderartsen dat je baby...,24 uur observatie van de pasgeborene,[{'division_url': 'https://www.azstlucas.be/sp...,Elke pasgeborene wordt - bij overnachting - bi...,[{'link_url': '#waarom-willen-we-dat-je-baby-m...,https://www.azstlucas.be/onderzoek-en-behandel...,12/11/2023 15:26:50,## Waarom willen de kinderartsen dat je baby m...,0
0,\n## Waarom willen de kinderartsen dat je baby...,24 uur observatie van de pasgeborene,[{'division_url': 'https://www.azstlucas.be/sp...,Elke pasgeborene wordt - bij overnachting - bi...,[{'link_url': '#waarom-willen-we-dat-je-baby-m...,https://www.azstlucas.be/onderzoek-en-behandel...,12/11/2023 15:26:50,## Mogelijke afwijkingen\n\n\n### Aangeboren h...,1
0,\n## Waarom willen de kinderartsen dat je baby...,24 uur observatie van de pasgeborene,[{'division_url': 'https://www.azstlucas.be/sp...,Elke pasgeborene wordt - bij overnachting - bi...,[{'link_url': '#waarom-willen-we-dat-je-baby-m...,https://www.azstlucas.be/onderzoek-en-behandel...,12/11/2023 15:26:50,### Infecties\n\n\nVerschillende infecties wor...,2
0,\n## Waarom willen de kinderartsen dat je baby...,24 uur observatie van de pasgeborene,[{'division_url': 'https://www.azstlucas.be/sp...,Elke pasgeborene wordt - bij overnachting - bi...,[{'link_url': '#waarom-willen-we-dat-je-baby-m...,https://www.azstlucas.be/onderzoek-en-behandel...,12/11/2023 15:26:50,### Aangeboren darmafwijkingen\n\n\nHet is pas...,3
0,\n## Waarom willen de kinderartsen dat je baby...,24 uur observatie van de pasgeborene,[{'division_url': 'https://www.azstlucas.be/sp...,Elke pasgeborene wordt - bij overnachting - bi...,[{'link_url': '#waarom-willen-we-dat-je-baby-m...,https://www.azstlucas.be/onderzoek-en-behandel...,12/11/2023 15:26:50,De kinderarts onderzoekt je baby in normale om...,4
